In [ ]:
!pip install colorama



In [ ]:
"""
A command-line task manager application using SQLite3 and an OOP structure.

Features:
- Add new tasks with descriptions, priority, and optional due dates.
- List all tasks, with color-coding, filtering, and sorting.
- Update a task's status (e.g., 'pending', 'in_progress', 'completed').
- Update a task's priority (1-High, 2-Medium, 3-Low).
- Update a task's due date.
- Delete a task by its ID.
- Data is persisted in an SQLite3 database file (`tasks.db`).
"""

import sqlite3
import argparse
import sys
from datetime import datetime

# Try to import colorama, fail gracefully if not installed
try:
    from colorama import Fore, Style, init
    init(autoreset=True)
except ImportError:
    print("Colorama not installed. Output will be plain. \nInstall with: pip install colorama", file=sys.stderr)
    # Create dummy Fore and Style classes
    class DummyColor:
        def __getattr__(self, name):
            return ""
    Fore = DummyColor()
    Style = DummyColor()

class TaskManager:
    """
    Manages tasks using an SQLite3 database.

    Handles connection, table creation, and all CRUD (Create, Read, Update, Delete)
    operations for tasks.
    """

    def __init__(self, db_file='tasks.db'):
        """
        Initialize the TaskManager and connect to the database.

        Args:
            db_file (str): The name of the database file to use.
        """
        try:
            self.conn = sqlite3.connect(db_file)
            self.cursor = self.conn.cursor()
            self._create_table()
        except sqlite3.Error as e:
            print(Fore.RED + f"Error connecting to database: {e}", file=sys.stderr)
            sys.exit(1)

    def _create_table(self):
        """
        Create the 'tasks' table if it doesn't already exist.

        The table stores:
        - id: A unique integer primary key.
        - description: The text of the task (cannot be null).
        - priority: An integer (1-High, 2-Medium, 3-Low). Defaults to 3.
        - status: The task's status ('pending', 'in_progress', 'completed'). Defaults to 'pending'.
        - created_at: The timestamp when the task was created.
        - due_date: An optional due date in 'YYYY-MM-DD' format.
        """
        try:
            self.cursor.execute("""
                CREATE TABLE IF NOT EXISTS tasks (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    description TEXT NOT NULL,
                    priority INTEGER DEFAULT 3,
                    status TEXT DEFAULT 'pending',
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    due_date TEXT
                )
            """)
            self.conn.commit()
        except sqlite3.Error as e:
            print(Fore.RED + f"Error creating table: {e}", file=sys.stderr)

    def _validate_date(self, date_str):
        """Helper to validate YYYY-MM-DD format."""
        if date_str is None:
            return True # None is allowed
        try:
            datetime.strptime(date_str, '%Y-%m-%d')
            return True
        except ValueError:
            return False

    def add_task(self, description, priority, due_date=None):
        """
        Add a new task to the database.

        Args:
            description (str): The text description of the task.
            priority (int): The priority of the task (1-3).
            due_date (str, optional): The due date in 'YYYY-MM-DD' format.
        """
        if not (1 <= priority <= 3):
            print(Fore.RED + "Error: Priority must be between 1 (High) and 3 (Low).", file=sys.stderr)
            return

        if not self._validate_date(due_date):
            print(Fore.RED + "Error: Due date must be in YYYY-MM-DD format.", file=sys.stderr)
            return

        try:
            sql = "INSERT INTO tasks (description, priority, due_date) VALUES (?, ?, ?)"
            self.cursor.execute(sql, (description, priority, due_date))
            self.conn.commit()
            print(Fore.GREEN + f"Added task: '{description}' with priority {priority} (ID: {self.cursor.lastrowid})")
        except sqlite3.Error as e:
            print(Fore.RED + f"Error adding task: {e}", file=sys.stderr)

    def delete_task(self, task_id):
        """
        Delete a task from the database by its ID.

        Args:
            task_id (int): The ID of the task to delete.
        """
        if not self._task_exists(task_id):
            print(Fore.RED + f"Error: No task found with ID {task_id}", file=sys.stderr)
            return

        try:
            self.cursor.execute("DELETE FROM tasks WHERE id = ?", (task_id,))
            self.conn.commit()
            print(Fore.GREEN + f"Deleted task with ID: {task_id}")
        except sqlite3.Error as e:
            print(Fore.RED + f"Error deleting task: {e}", file=sys.stderr)

    def _task_exists(self, task_id):
        """Check if a task with the given ID exists."""
        self.cursor.execute("SELECT 1 FROM tasks WHERE id = ?", (task_id,))
        return self.cursor.fetchone() is not None

    def update_task_status(self, task_id, status):
        """
        Update the status of an existing task.

        Args:
            task_id (int): The ID of the task to update.
            status (str): The new status ('pending', 'in_progress', 'completed').
        """
        valid_statuses = ['pending', 'in_progress', 'completed']
        if status not in valid_statuses:
            print(Fore.RED + f"Error: Invalid status. Choose from: {', '.join(valid_statuses)}", file=sys.stderr)
            return

        if not self._task_exists(task_id):
            print(Fore.RED + f"Error: No task found with ID {task_id}", file=sys.stderr)
            return

        try:
            self.cursor.execute("UPDATE tasks SET status = ? WHERE id = ?", (status, task_id))
            self.conn.commit()
            print(Fore.GREEN + f"Updated task ID {task_id} to status: {status}")
        except sqlite3.Error as e:
            print(Fore.RED + f"Error updating task status: {e}", file=sys.stderr)

    def update_task_priority(self, task_id, priority):
        """
        Update the priority of an existing task.

        Args:
            task_id (int): The ID of the task to update.
            priority (int): The new priority (1-3).
        """
        if not (1 <= priority <= 3):
            print(Fore.RED + "Error: Priority must be between 1 (High) and 3 (Low).", file=sys.stderr)
            return

        if not self._task_exists(task_id):
            print(Fore.RED + f"Error: No task found with ID {task_id}", file=sys.stderr)
            return

        try:
            self.cursor.execute("UPDATE tasks SET priority = ? WHERE id = ?", (priority, task_id))
            self.conn.commit()
            print(Fore.GREEN + f"Updated task ID {task_id} to priority: {priority}")
        except sqlite3.Error as e:
            print(Fore.RED + f"Error updating task priority: {e}", file=sys.stderr)

    def update_task_due_date(self, task_id, due_date):
        """
        Update the due date of an existing task.

        Args:
            task_id (int): The ID of the task to update.
            due_date (str): The new due date in 'YYYY-MM-DD' format or 'none'.
        """
        if not self._validate_date(due_date) and due_date.lower() != 'none':
             print(Fore.RED + "Error: Due date must be in YYYY-MM-DD format or 'none' to remove.", file=sys.stderr)
             return

        if not self._task_exists(task_id):
            print(Fore.RED + f"Error: No task found with ID {task_id}", file=sys.stderr)
            return

        date_to_set = None if due_date.lower() == 'none' else due_date

        try:
            self.cursor.execute("UPDATE tasks SET due_date = ? WHERE id = ?", (date_to_set, task_id))
            self.conn.commit()
            print(Fore.GREEN + f"Updated task ID {task_id} due date to: {date_to_set}")
        except sqlite3.Error as e:
            print(Fore.RED + f"Error updating due date: {e}", file=sys.stderr)

    def _get_color(self, priority, status, due_date_str):
        """Get the color for a task based on its properties."""
        if status == 'completed':
            return Fore.GREEN + Style.DIM

        # Check for overdue
        if due_date_str:
            try:
                due_date = datetime.strptime(due_date_str, '%Y-%m-%d').date()
                if due_date < datetime.now().date():
                    return Fore.RED + Style.BRIGHT # Overdue
            except ValueError:
                pass # Should not happen if _validate_date is used

        if status == 'in_progress':
            return Fore.YELLOW + Style.BRIGHT

        if priority == 1: # High
            return Fore.RED
        if priority == 2: # Medium
            return Fore.YELLOW
        if priority == 3: # Low
            return Fore.WHITE

        return Fore.WHITE # Default

    def list_tasks(self, status_filter=None, sort_by=None):
        """
        List all tasks, optionally filtering by status and sorting.
        Tasks are sorted by priority (ascending) and then by creation date.

        Args:
            status_filter (str, optional): If provided, only show tasks with this status.
            sort_by (str, optional): Field to sort by (e.g., 'priority', 'due_date').
        """
        try:
            sql = "SELECT id, description, priority, status, created_at, due_date FROM tasks"
            params = []

            if status_filter:
                valid_statuses = ['pending', 'in_progress', 'completed']
                if status_filter not in valid_statuses:
                    print(Fore.RED + f"Error: Invalid status filter. Choose from: {', '.join(valid_statuses)}", file=sys.stderr)
                    return
                sql += " WHERE status = ?"
                params.append(status_filter)

            # Whitelist valid sort columns
            valid_sort_cols = ['id', 'priority', 'status', 'created_at', 'due_date']
            if sort_by:
                if sort_by in valid_sort_cols:
                    # Handle NULLs in due_date sorting
                    if sort_by == 'due_date':
                         sql += " ORDER BY CASE WHEN due_date IS NULL THEN 1 ELSE 0 END, due_date ASC"
                    else:
                        sql += f" ORDER BY {sort_by} ASC"
                else:
                    print(Fore.YELLOW + f"Warning: Invalid sort key '{sort_by}'. Using default sort.")
                    sql += " ORDER BY priority ASC, created_at ASC"
            else:
                # Default sort
                sql += " ORDER BY priority ASC, created_at ASC"

            self.cursor.execute(sql, params)
            tasks = self.cursor.fetchall()

            if not tasks:
                if status_filter:
                    print(Fore.YELLOW + f"No tasks found with status '{status_filter}'.")
                else:
                    print(Fore.YELLOW + "No tasks found. Add one with the 'add' command!")
                return

            # Helper for formatting
            pri_map = {1: 'High', 2: 'Medium', 3: 'Low'}

            print(Style.BRIGHT + "\n--- Task List ---")
            print(f"{'ID':<4} | {'Priority':<8} | {'Status':<12} | {'Due Date':<11} | {'Created':<16} | {'Description':<40}")
            print("-" * 100)

            for task in tasks:
                task_id, desc, priority, status, created, due_date = task

                # Get color
                color = self._get_color(priority, status, due_date)

                # Format fields
                created_dt = datetime.strptime(created, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d %H:%M')
                due_str = due_date if due_date else '---'
                pri_str = pri_map.get(priority, 'N/A')

                print(color + f"{task_id:<4} | {pri_str:<8} | {status:<12} | {due_str:<11} | {created_dt:<16} | {desc:<40}")

            print("-" * 100)

        except sqlite3.Error as e:
            print(Fore.RED + f"Error listing tasks: {e}", file=sys.stderr)

    def __del__(self):
        """
        Destructor to ensure the database connection is closed
        when the object is garbage collected.
        """
        if hasattr(self, 'conn') and self.conn:
            self.conn.close()


def valid_date_type(s):
    """Argparse type for validating date strings."""
    try:
        return datetime.strptime(s, '%Y-%m-%d').strftime('%Y-%m-%d')
    except ValueError:
        msg = f"Not a valid date: '{s}'. Expected YYYY-MM-DD."
        raise argparse.ArgumentTypeError(msg)

def main(argv=None):
    """
    Main function to parse command-line arguments and run the TaskManager.
    Accepts an optional list of arguments for interactive use.
    """
    parser = argparse.ArgumentParser(
        description="A command-line task manager.",
        formatter_class=argparse.RawTextHelpFormatter
    )
    subparsers = parser.add_subparsers(dest='command', help='Available commands', required=True)

    # --- 'add' command ---
    add_parser = subparsers.add_parser('add', help='Add a new task')
    add_parser.add_argument('description', type=str, help='The description of the task')
    add_parser.add_argument('-p', '--priority', type=int, default=3, choices=[1, 2, 3],
                            help='Task priority (1: High, 2: Medium, 3: Low). Default is 3.')
    add_parser.add_argument('-d', '--due', type=valid_date_type, help='Due date in YYYY-MM-DD format.')

    # --- 'list' command ---
    list_parser = subparsers.add_parser('list', help='List all tasks')
    list_parser.add_argument('-s', '--status', type=str, choices=['pending', 'in_progress', 'completed'],
                             help='Filter tasks by status')
    list_parser.add_argument('--sort', type=str, choices=['id', 'priority', 'status', 'created_at', 'due_date'],
                             help='Sort tasks by a specific field.')

    # --- 'update-status' command ---
    update_status_parser = subparsers.add_parser('update-status', help='Update a task\'s status')
    update_status_parser.add_argument('id', type=int, help='The ID of the task to update')
    update_status_parser.add_argument('status', type=str, choices=['pending', 'in_progress', 'completed'],
                                    help='The new status for the task')

    # --- 'update-priority' command ---
    update_pri_parser = subparsers.add_parser('update-priority', help='Update a task\'s priority')
    update_pri_parser.add_argument('id', type=int, help='The ID of the task to update')
    update_pri_parser.add_argument('priority', type=int, choices=[1, 2, 3],
                                 help='The new priority for the task (1-3)')

    # --- 'update-due' command ---
    update_due_parser = subparsers.add_parser('update-due', help='Update a task\'s due date')
    update_due_parser.add_argument('id', type=int, help='The ID of the task to update')
    update_due_parser.add_argument('date', type=str,
                                 help="The new due date (YYYY-MM-DD) or 'none' to remove.")

    # --- 'delete' command ---
    delete_parser = subparsers.add_parser('delete', help='Delete a task by its ID')
    delete_parser.add_argument('id', type=int, help='The ID of the task to delete')

    # Parse the arguments
    try:
        # Use provided argv list if available, otherwise use sys.argv[1:]
        args = parser.parse_args(argv)
    except argparse.ArgumentTypeError as e:
        print(Fore.RED + f"Argument Error: {e}", file=sys.stderr)
        return # Changed sys.exit(1) to return for interactive use
    except SystemExit as e:
        # argparse raises SystemExit on errors (like missing required arguments)
        # and when --help is requested. We should gracefully handle it for Colab.
        if e.code == 0: # This usually means --help was called successfully
            # If it's a successful exit (e.g., --help), we just return.
            return
        else: # This means there was an error parsing arguments.
            # argparse will have already printed an error message to stderr.
            # We just return to prevent the Colab kernel from stopping.
            return

    # Initialize the manager
    manager = TaskManager()

    # Execute the corresponding command
    try:
        if args.command == 'add':
            manager.add_task(args.description, args.priority, args.due)
        elif args.command == 'list':
            manager.list_tasks(args.status, args.sort)
        elif args.command == 'update-status':
            manager.update_task_status(args.id, args.status)
        elif args.command == 'update-priority':
            manager.update_task_priority(args.id, args.priority)
        elif args.command == 'update-due':
            manager.update_task_due_date(args.id, args.date)
        elif args.command == 'delete':
            manager.delete_task(args.id)
    except Exception as e:
        print(Fore.RED + f"An unexpected error occurred: {e}", file=sys.stderr)
        # Not using sys.exit(1) here either, to allow continued interactive use.
        return

if __name__ == "__main__":
    # In a Colab notebook, direct execution of a cell that defines main()
    # can lead to issues with argparse trying to parse kernel arguments
    # from sys.argv. To run commands, explicitly call main() with a list
    # of arguments, like: main(['list']) or main(['add', 'Description']).
    # This block is now effectively a placeholder for direct script execution
    # but does nothing in Colab by default, forcing explicit calls.
    pass


Colorama not installed. Output will be plain. 
Install with: pip install colorama


### Add a new task

To add a new task, you can call the `main` function with the 'add' command, followed by the description and optional priority and due date.

In [ ]:
main(['add', 'Buy groceries', '-p', '1', '-d', '2023-12-31'])
main(['add', 'Finish report', '-p', '2'])
main(['add', 'Call Mom', '-d', '2023-12-25'])
main(['add', 'Review code'])


Added task: 'Buy groceries' with priority 1 (ID: 1)
Added task: 'Finish report' with priority 2 (ID: 2)
Added task: 'Call Mom' with priority 3 (ID: 3)
Added task: 'Review code' with priority 3 (ID: 4)


### List all tasks

To list all tasks, call the `main` function with the 'list' command. You can also filter by status or sort by different fields.

In [ ]:
!python3 task_manager.py -h


python3: can't open file '/content/task_manager.py': [Errno 2] No such file or directory


### List tasks with a specific status

To list tasks with a specific status, use the `-s` or `--status` flag.

In [ ]:
main(['list', '-s', 'pending'])



--- Task List ---
ID   | Priority | Status       | Due Date    | Created          | Description                             
----------------------------------------------------------------------------------------------------
1    | High     | pending      | 2023-12-31  | 2025-11-10 02:42 | Buy groceries                           
2    | Medium   | pending      | ---         | 2025-11-10 02:42 | Finish report                           
3    | Low      | pending      | 2023-12-25  | 2025-11-10 02:42 | Call Mom                                
4    | Low      | pending      | ---         | 2025-11-10 02:42 | Review code                             
----------------------------------------------------------------------------------------------------


### Update a task's status

To update a task's status, use the `update-status` command with the task ID and the new status.

In [ ]:
main(['update-status', '1', 'completed'])
main(['list'])


Updated task ID 1 to status: completed

--- Task List ---
ID   | Priority | Status       | Due Date    | Created          | Description                             
----------------------------------------------------------------------------------------------------
1    | High     | completed    | 2023-12-31  | 2025-11-10 02:42 | Buy groceries                           
2    | Medium   | pending      | ---         | 2025-11-10 02:42 | Finish report                           
3    | Low      | pending      | 2023-12-25  | 2025-11-10 02:42 | Call Mom                                
4    | Low      | pending      | ---         | 2025-11-10 02:42 | Review code                             
----------------------------------------------------------------------------------------------------
